In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from loguru import logger

logger.enable("probafcst")

In [ ]:
import pandas as pd
import seaborn as sns

from probafcst.backtest import backtest

sns.set_theme(style="ticks")

In [ ]:
quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]

In [ ]:
bikes = pd.read_parquet("../data/bikes.parquet")
load = pd.read_parquet("../data/energy.parquet")
load = load.asfreq("h").dropna()
bikes = bikes.asfreq("D").dropna()

In [ ]:
debug = True

if debug:
    bikes = bikes.loc["2024":]
    load = load.loc["2024":]

In [ ]:
# predictions are made on wednesdays
eval_start = "2024-10-23"
# eval_end = "2025-02-09"

# compute the length of the training window
n_train_bikes = bikes.loc[:eval_start].shape[0]
n_train_energy = load.loc[:eval_start].shape[0]
n_train_energy

## Backtesting


In [ ]:
import dvc.api
from omegaconf import OmegaConf

from probafcst.models import get_model

In [ ]:
def backtest_single(model_name: str, target: str):
    """Backtest single model on challenge weeks."""
    quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]
    pipe_params = dvc.api.params_show()
    model_params = OmegaConf.create(pipe_params).train[target]
    model_params.selected = model_name
    model = get_model(model_params, quantiles=quantile_levels, n_jobs=1)

    use_bikes = target == "bikes"

    if use_bikes:
        X = bikes.drop(columns="bike_count")
        y = bikes["bike_count"]
        initial_window = n_train_bikes
        DAY_HOURS = 1
    else:
        X = load.drop(columns="load")
        y = load["load"]
        initial_window = n_train_energy
        DAY_HOURS = 24

    if model_name == "quantreg":
        # use less data for quantreg
        X = X.loc["2021":]
        y = y.loc["2021":]

    results, metrics, predictions, add_metrics = backtest(
        model,
        y=y,
        X=X,
        forecast_steps=DAY_HOURS * 7,  # one week
        quantiles=quantile_levels,
        initial_window=initial_window,
        step_length=DAY_HOURS * 7,  # one week
        backend="loky",
    )
    return results, metrics, predictions, add_metrics

In [ ]:
def backtest_all(models: list | None = None):
    """Backtest all models."""
    targets = ["bikes", "energy"]
    if models is None:
        models = ["benchmark", "quantreg", "xgb-custom", "lgbm", "catboost"]
    metrics = {target: {model_name: {} for model_name in models} for target in targets}
    for target in targets:
        for model_name in models:
            _, metrics[target][model_name], _, _ = backtest_single(model_name, target)
            pl = metrics[target][model_name]["pinball_loss"]
            cov_50 = metrics[target][model_name]["coverage_50"]
            cov_95 = metrics[target][model_name]["coverage_95"]
            print(
                f"{target:<8} {model_name} | Pinball loss    = {pl['mean']:.2f} (± {pl['std']:.2f})"
            )
            print(
                f"{target:<8} {model_name} | 50% PI Coverage = {cov_50['mean']:.2f} (± {cov_50['std']:.2f})"
            )
            print(
                f"{target:<8} {model_name} | 95% PI Coverage = {cov_95['mean']:.2f} (± {cov_95['std']:.2f})"
            )

    return metrics

In [ ]:
result = backtest_all(models=["lgbm"])